In [4]:
from pprint import pprint
import requests

In [6]:
# Jambase API Key
jambase='961d6740-8d84-4af8-847c-87174bec671c'

import http.client

conn = http.client.HTTPSConnection("www.jambase.com")

headers = { 'Accept': "application/json" }

conn.request("GET", f"/jb-api/v1/geographies/cities?geoCityName=San+Antonio&geoCountryIso2=US&cityHasUpcomingEvents=true&apikey={jambase}", headers=headers)

res = conn.getresponse()
data = res.read()

pprint(data.decode("utf-8"))

('{"success":true,"pagination":{"page":1,"perPage":40,"totalItems":1,"totalPages":1},"cities":[{"@type":"City","identifier":"jambase:4219129","name":"San '
 'Antonio","geo":{"@type":"GeoCoordinates","latitude":29.4722,"longitude":-98.5247},"address":{"@type":"PostalAddress","addressRegion":"US-TX","addressCountry":"US"},"containedInPlace":{"@type":"AdministrativeArea","identifier":"jambase:61","name":"San '
 'Antonio '
 'Area","geo":{"type":"GeoCoordinates","latitude":29.4722,"longitude":-98.5247},"address":{"@type":"PostalAddress","addressRegion":"US-TX","addressCountry":"US"},"containsPlace":[],"x-primaryCityId":"jambase:4219129","x-numUpcomingEvents":289},"x-timezone":"America\\/Chicago","x-numUpcomingEvents":243}],"request":{"endpoint":"https:\\/\\/www.jambase.com\\/jb-api\\/v1\\/geographies\\/cities","method":"GET","params":{"geoCityName":"San '
 'Antonio","geoCountryIso2":"US","cityHasUpcomingEvents":"true","apikey":"961d6740-8d84-4af8-847c-87174bec671c"},"ip":"2600:1700:1474:709

In [3]:
# Here's the same request in cURL

#curl --request GET \
#  --url 'https://www.jambase.com/jb-api/v1/events/id/axs:eventId?apikey=123' \
#  --header 'Accept: application/json'

In [13]:
# Spotify API request:

#IMPORTANT: API resets after one hour of use!

import requests

url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = {
    "grant_type": "client_credentials",
    "client_id": "22ba93d78b104ade8f90c67ec7cd1e35",
    "client_secret": "7c3fb228e7d94fd7808614993728843e"
}

response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    access_token = response.json()["access_token"]
    print("Access Token:", access_token)
    #access_token='Bearer '+ access_token
else:
    print("Failed to retrieve access token. Status code:", response.status_code)



Access Token: BQAPCJDXhE2mfR2tmNQXPeEi8NyNjdMhN58WBViudEV92XzVWUOF1L971WOUW9rqV3AMVkqoBWXgUzX_3oNyhF3PJFW-vWTYnrc_9affipD1FzxiYLk


In [14]:
import random
import string

# Define the client_id, redirect_uri, and scope
client_id = '22ba93d78b104ade8f90c67ec7cd1e35'
redirect_uri = 'http://localhost:8080'
scope = 'user-follow-read'

# Generate a random state string
stateKey = 'state_key'  # Assuming you have a variable for stateKey
state = ''.join(random.choice(string.ascii_letters) for _ in range(16))
# You can use the 'stateKey' variable for localStorage as well, similar to the original code

# Build the URL
url = 'https://accounts.spotify.com/authorize'
url += '?response_type=token'
url += f'&client_id={client_id}'
url += f'&scope={scope}'
url += f'&redirect_uri={redirect_uri}'
url += f'&state={state}'

print(url)

https://accounts.spotify.com/authorize?response_type=token&client_id=22ba93d78b104ade8f90c67ec7cd1e35&scope=user-follow-read&redirect_uri=http://localhost:8080&state=MkADDDvZJcPSWikr


In [12]:
# Define the URL and headers
url='https://api.spotify.com/v1/me/following?type=artist'
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Make the GET request using the requests library
response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Request was successful
    data = response.json()  # Parse the response as JSON
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)  # Print the response content for debugging


Request failed with status code: 403
{
  "error" : {
    "status" : 403,
    "message" : "Insufficient client scope"
  }
}
